In [ ]:
!pip install Pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for Pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=431d7d022a43572c644b076ab46cc613151c2ef9224ac61292739729a3194c26
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built Pyspark


In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *

In [ ]:
help(window)

Help on function window in module pyspark.sql.functions:

window(timeColumn: 'ColumnOrName', windowDuration: str, slideDuration: Optional[str] = None, startTime: Optional[str] = None) -> pyspark.sql.column.Column
    Bucketize rows into one or more time windows given a timestamp specifying column. Window
    starts are inclusive but the window ends are exclusive, e.g. 12:05 will be in the window
    [12:05,12:10) but not in [12:00,12:05). Windows can support microsecond precision. Windows in
    the order of months are not supported.
    
    The time column must be of :class:`pyspark.sql.types.TimestampType`.
    
    Durations are provided as strings, e.g. '1 second', '1 day 12 hours', '2 minutes'. Valid
    interval strings are 'week', 'day', 'hour', 'minute', 'second', 'millisecond', 'microsecond'.
    If the ``slideDuration`` is not provided, the windows will be tumbling windows.
    
    The startTime is the offset with respect to 1970-01-01 00:00:00 UTC with which to start
    w

In [ ]:
spark=SparkSession.builder.appName("test").getOrCreate()

In [ ]:
data= [("James","Sales","NY",9000,34),
       ("Alicia","Sales","NY",8600,56),
       ("Robert","Sales","CA",8100,30),
       ("John","Sales","AZ",8600,31),
       ("Ross","Sales","AZ",8100,33),
       ("Kathy","Sales","az",1000,39),
       ("Lisa","Finance","CA",9000,24),
       ("Deja","Finance","CA",9900,40),
       ("Sugie","Finance","NY",8300,36),
       ("Ram","Finance","NY",7900,53),
       ("Sathya","Finance","AZ",8200,53),
       ("Kyle","Marketing","CA",8000,25),
       ("Reid","Marketing","NY",9100,50)]

In [ ]:
schema= 'empname string,dept string,state string,salary int,age int'

In [ ]:
df=spark.createDataFrame(data=data,schema = schema)
df.show()

+-------+---------+-----+------+---+
|empname|     dept|state|salary|age|
+-------+---------+-----+------+---+
|  James|    Sales|   NY|  9000| 34|
| Alicia|    Sales|   NY|  8600| 56|
| Robert|    Sales|   CA|  8100| 30|
|   John|    Sales|   AZ|  8600| 31|
|   Ross|    Sales|   AZ|  8100| 33|
|  Kathy|    Sales|   az|  1000| 39|
|   Lisa|  Finance|   CA|  9000| 24|
|   Deja|  Finance|   CA|  9900| 40|
|  Sugie|  Finance|   NY|  8300| 36|
|    Ram|  Finance|   NY|  7900| 53|
| Sathya|  Finance|   AZ|  8200| 53|
|   Kyle|Marketing|   CA|  8000| 25|
|   Reid|Marketing|   NY|  9100| 50|
+-------+---------+-----+------+---+



In [ ]:
df.dtypes

[('empname', 'string'),
 ('dept', 'string'),
 ('state', 'string'),
 ('salary', 'int'),
 ('age', 'int')]

In [ ]:
#Ranking Functions()

In [ ]:
spec=Window.partitionBy(df.dept).orderBy(df.salary.desc())

In [ ]:
spec

In [ ]:
df.select(df.dept,df.salary).show()

+---------+------+
|     dept|salary|
+---------+------+
|    Sales|  9000|
|    Sales|  8600|
|    Sales|  8100|
|    Sales|  8600|
|    Sales|  8100|
|    Sales|  1000|
|  Finance|  9000|
| Finanace|  9900|
|  Finance|  8300|
|  Finance|  7900|
|  Finance|  8200|
|Marketing|  8000|
|Marketing|  9100|
+---------+------+



In [ ]:
 #Row_number()
 df.select(df.dept,df.salary).withColumn("row_number_rank",row_number().over(spec)).show()

+---------+------+---------------+
|     dept|salary|row_number_rank|
+---------+------+---------------+
|  Finance|  7900|              1|
|  Finance|  8200|              2|
|  Finance|  8300|              3|
|  Finance|  9000|              4|
|  Finance|  9900|              5|
|Marketing|  8000|              1|
|Marketing|  9100|              2|
|    Sales|  1000|              1|
|    Sales|  8100|              2|
|    Sales|  8100|              3|
|    Sales|  8600|              4|
|    Sales|  8600|              5|
|    Sales|  9000|              6|
+---------+------+---------------+



In [ ]:
#Rank()
df.select(df.dept,df.salary).withColumn("row_number_rank",row_number().over(spec))\
 .withColumn("rank",rank().over(spec)).show()

+---------+------+---------------+----+
|     dept|salary|row_number_rank|rank|
+---------+------+---------------+----+
|  Finance|  7900|              1|   1|
|  Finance|  8200|              2|   2|
|  Finance|  8300|              3|   3|
|  Finance|  9000|              4|   4|
|  Finance|  9900|              5|   5|
|Marketing|  8000|              1|   1|
|Marketing|  9100|              2|   2|
|    Sales|  1000|              1|   1|
|    Sales|  8100|              2|   2|
|    Sales|  8100|              3|   2|
|    Sales|  8600|              4|   4|
|    Sales|  8600|              5|   4|
|    Sales|  9000|              6|   6|
+---------+------+---------------+----+



In [ ]:
#Dense_Rank()
df.select(df.dept,df.salary).withColumn("row_number_rank",row_number().over(spec))\
 .withColumn("rank",rank().over(spec))\
 .withColumn("dense_rank",dense_rank().over(spec)).show()

+---------+------+---------------+----+----------+
|     dept|salary|row_number_rank|rank|dense_rank|
+---------+------+---------------+----+----------+
|  Finance|  7900|              1|   1|         1|
|  Finance|  8200|              2|   2|         2|
|  Finance|  8300|              3|   3|         3|
|  Finance|  9000|              4|   4|         4|
|  Finance|  9900|              5|   5|         5|
|Marketing|  8000|              1|   1|         1|
|Marketing|  9100|              2|   2|         2|
|    Sales|  1000|              1|   1|         1|
|    Sales|  8100|              2|   2|         2|
|    Sales|  8100|              3|   2|         2|
|    Sales|  8600|              4|   4|         3|
|    Sales|  8600|              5|   4|         3|
|    Sales|  9000|              6|   6|         4|
+---------+------+---------------+----+----------+



In [ ]:
#percentile_rank()

df.select(df.dept,df.salary).withColumn("row_number_rank",row_number().over(spec))\
 .withColumn("rank",rank().over(spec))\
 .withColumn("dense_rank",dense_rank().over(spec))\
 .withColumn("percent_rank",percent_rank().over(spec)).show()

+---------+------+---------------+----+----------+------------+
|     dept|salary|row_number_rank|rank|dense_rank|percent_rank|
+---------+------+---------------+----+----------+------------+
|  Finance|  7900|              1|   1|         1|         0.0|
|  Finance|  8200|              2|   2|         2|        0.25|
|  Finance|  8300|              3|   3|         3|         0.5|
|  Finance|  9000|              4|   4|         4|        0.75|
|  Finance|  9900|              5|   5|         5|         1.0|
|Marketing|  8000|              1|   1|         1|         0.0|
|Marketing|  9100|              2|   2|         2|         1.0|
|    Sales|  1000|              1|   1|         1|         0.0|
|    Sales|  8100|              2|   2|         2|         0.2|
|    Sales|  8100|              3|   2|         2|         0.2|
|    Sales|  8600|              4|   4|         3|         0.6|
|    Sales|  8600|              5|   4|         3|         0.6|
|    Sales|  9000|              6|   6| 

In [ ]:
#ntile()

df.select(df.dept,df.salary).withColumn("row_number_rank",row_number().over(spec))\
 .withColumn("rank",rank().over(spec))\
 .withColumn("dense_rank",dense_rank().over(spec))\
 .withColumn("percent_rank",percent_rank().over(spec))\
 .withColumn("ntile_rank",ntile(6).over(spec)).show()

+---------+------+---------------+----+----------+------------+----------+
|     dept|salary|row_number_rank|rank|dense_rank|percent_rank|ntile_rank|
+---------+------+---------------+----+----------+------------+----------+
|  Finance|  7900|              1|   1|         1|         0.0|         1|
|  Finance|  8200|              2|   2|         2|        0.25|         2|
|  Finance|  8300|              3|   3|         3|         0.5|         3|
|  Finance|  9000|              4|   4|         4|        0.75|         4|
|  Finance|  9900|              5|   5|         5|         1.0|         5|
|Marketing|  8000|              1|   1|         1|         0.0|         1|
|Marketing|  9100|              2|   2|         2|         1.0|         2|
|    Sales|  1000|              1|   1|         1|         0.0|         1|
|    Sales|  8100|              2|   2|         2|         0.2|         2|
|    Sales|  8100|              3|   2|         2|         0.2|         3|
|    Sales|  8600|       

In [ ]:
#ntile()

df.select(df.dept,df.salary).withColumn("row_number_rank",row_number().over(spec))\
 .withColumn("rank",rank().over(spec))\
 .withColumn("dense_rank",dense_rank().over(spec))\
 .withColumn("percent_rank",percent_rank().over(spec))\
 .withColumn("ntile_rank",ntile(3).over(spec)).show()

+---------+------+---------------+----+----------+------------+----------+
|     dept|salary|row_number_rank|rank|dense_rank|percent_rank|ntile_rank|
+---------+------+---------------+----+----------+------------+----------+
|  Finance|  7900|              1|   1|         1|         0.0|         1|
|  Finance|  8200|              2|   2|         2|        0.25|         1|
|  Finance|  8300|              3|   3|         3|         0.5|         2|
|  Finance|  9000|              4|   4|         4|        0.75|         2|
|  Finance|  9900|              5|   5|         5|         1.0|         3|
|Marketing|  8000|              1|   1|         1|         0.0|         1|
|Marketing|  9100|              2|   2|         2|         1.0|         2|
|    Sales|  1000|              1|   1|         1|         0.0|         1|
|    Sales|  8100|              2|   2|         2|         0.2|         1|
|    Sales|  8100|              3|   2|         2|         0.2|         2|
|    Sales|  8600|       

In [ ]:
#cummulative_distribution_Rank
df.select(df.dept,df.salary).withColumn("cum_dist_rank",cume_dist().over(spec)).show()

+---------+------+-------------------+
|     dept|salary|      cum_dist_rank|
+---------+------+-------------------+
|  Finance|  7900|                0.2|
|  Finance|  8200|                0.4|
|  Finance|  8300|                0.6|
|  Finance|  9000|                0.8|
|  Finance|  9900|                1.0|
|Marketing|  8000|                0.5|
|Marketing|  9100|                1.0|
|    Sales|  1000|0.16666666666666666|
|    Sales|  8100|                0.5|
|    Sales|  8100|                0.5|
|    Sales|  8600| 0.8333333333333334|
|    Sales|  8600| 0.8333333333333334|
|    Sales|  9000|                1.0|
+---------+------+-------------------+



In [ ]:
#lag()
#lag([column],offset_val,[alternate Result])
df.select(df.dept,df.salary).withColumn("Lag_prev_sal",lag("salary",1,0).over(spec)).show()

+---------+------+------------+
|     dept|salary|Lag_prev_sal|
+---------+------+------------+
|  Finance|  7900|           0|
|  Finance|  8200|        7900|
|  Finance|  8300|        8200|
|  Finance|  9000|        8300|
|  Finance|  9900|        9000|
|Marketing|  8000|           0|
|Marketing|  9100|        8000|
|    Sales|  1000|           0|
|    Sales|  8100|        1000|
|    Sales|  8100|        8100|
|    Sales|  8600|        8100|
|    Sales|  8600|        8600|
|    Sales|  9000|        8600|
+---------+------+------------+



In [ ]:
#lag()
#lag([column],offset_val,[alternate Result])
df.select(df.dept,df.salary).withColumn("Lag_prev_sal",lag("salary",1).over(spec)).show()

+---------+------+------------+
|     dept|salary|Lag_prev_sal|
+---------+------+------------+
|  Finance|  7900|        NULL|
|  Finance|  8200|        7900|
|  Finance|  8300|        8200|
|  Finance|  9000|        8300|
|  Finance|  9900|        9000|
|Marketing|  8000|        NULL|
|Marketing|  9100|        8000|
|    Sales|  1000|        NULL|
|    Sales|  8100|        1000|
|    Sales|  8100|        8100|
|    Sales|  8600|        8100|
|    Sales|  8600|        8600|
|    Sales|  9000|        8600|
+---------+------+------------+



In [ ]:
#lead()

df.select(df.dept,df.salary).withColumn("Lead_sal",lead("salary",1,0).over(spec)).show()

+---------+------+--------+
|     dept|salary|Lead_sal|
+---------+------+--------+
|  Finance|  7900|    8200|
|  Finance|  8200|    8300|
|  Finance|  8300|    9000|
|  Finance|  9000|    9900|
|  Finance|  9900|       0|
|Marketing|  8000|    9100|
|Marketing|  9100|       0|
|    Sales|  1000|    8100|
|    Sales|  8100|    8100|
|    Sales|  8100|    8600|
|    Sales|  8600|    8600|
|    Sales|  8600|    9000|
|    Sales|  9000|       0|
+---------+------+--------+



In [ ]:

df.select(df.dept,df.salary).withColumn("Lag_prev_sal",lag("salary",1,0).over(spec))\
.withColumn("Lead_sal",lead("salary",1,0).over(spec)).show()

+---------+------+------------+--------+
|     dept|salary|Lag_prev_sal|Lead_sal|
+---------+------+------------+--------+
|  Finance|  7900|           0|    8200|
|  Finance|  8200|        7900|    8300|
|  Finance|  8300|        8200|    9000|
|  Finance|  9000|        8300|    9900|
|  Finance|  9900|        9000|       0|
|Marketing|  8000|           0|    9100|
|Marketing|  9100|        8000|       0|
|    Sales|  1000|           0|    8100|
|    Sales|  8100|        1000|    8100|
|    Sales|  8100|        8100|    8600|
|    Sales|  8600|        8100|    8600|
|    Sales|  8600|        8600|    9000|
|    Sales|  9000|        8600|       0|
+---------+------+------------+--------+



In [ ]:
#aggregate window function
df.select(df.dept,df.salary).withColumn("Sum_sal_dept",sum("salary").over(spec)).show()

+---------+------+------------+
|     dept|salary|Sum_sal_dept|
+---------+------+------------+
|  Finance|  9900|        9900|
|  Finance|  9000|       18900|
|  Finance|  8300|       27200|
|  Finance|  8200|       35400|
|  Finance|  7900|       43300|
|Marketing|  9100|        9100|
|Marketing|  8000|       17100|
|    Sales|  9000|        9000|
|    Sales|  8600|       26200|
|    Sales|  8600|       26200|
|    Sales|  8100|       42400|
|    Sales|  8100|       42400|
|    Sales|  1000|       43400|
+---------+------+------------+



In [ ]:
spec=Window.partitionBy(df.dept)

In [ ]:

df.select(df.dept,df.salary).withColumn("Sum_sal_dept",sum("salary").over(spec)).show()

+---------+------+------------+
|     dept|salary|Sum_sal_dept|
+---------+------+------------+
|  Finance|  9000|       43300|
|  Finance|  9900|       43300|
|  Finance|  8300|       43300|
|  Finance|  7900|       43300|
|  Finance|  8200|       43300|
|Marketing|  8000|       17100|
|Marketing|  9100|       17100|
|    Sales|  9000|       43400|
|    Sales|  8600|       43400|
|    Sales|  8100|       43400|
|    Sales|  8600|       43400|
|    Sales|  8100|       43400|
|    Sales|  1000|       43400|
+---------+------+------------+



In [ ]:
#first ()
spec=Window.partitionBy(df.dept).orderBy(df.salary.desc())

df.select(df.dept,df.salary).withColumn("Highest_Sal",first("salary").over(spec)).show()

+---------+------+-----------+
|     dept|salary|Highest_Sal|
+---------+------+-----------+
|  Finance|  9900|       9900|
|  Finance|  9000|       9900|
|  Finance|  8300|       9900|
|  Finance|  8200|       9900|
|  Finance|  7900|       9900|
|Marketing|  9100|       9100|
|Marketing|  8000|       9100|
|    Sales|  9000|       9000|
|    Sales|  8600|       9000|
|    Sales|  8600|       9000|
|    Sales|  8100|       9000|
|    Sales|  8100|       9000|
|    Sales|  1000|       9000|
+---------+------+-----------+



In [ ]:
#last ()
spec=Window.partitionBy(df.dept)

df.select(df.dept,df.salary).withColumn("Lowest_Sal",last("salary").over(spec)).show()

+---------+------+----------+
|     dept|salary|Lowest_Sal|
+---------+------+----------+
|  Finance|  9000|      8200|
|  Finance|  9900|      8200|
|  Finance|  8300|      8200|
|  Finance|  7900|      8200|
|  Finance|  8200|      8200|
|Marketing|  8000|      9100|
|Marketing|  9100|      9100|
|    Sales|  9000|      1000|
|    Sales|  8600|      1000|
|    Sales|  8100|      1000|
|    Sales|  8600|      1000|
|    Sales|  8100|      1000|
|    Sales|  1000|      1000|
+---------+------+----------+

